# Simulation with default parameters and periodic boundary conditions on slab mesh

The following notebook outlines a sample simulation of the perfusion and gas exchange model on a periodic mesh mimicking an infinite sheet positioned between alveoli (an approximation to the 2D problem, collapsing the $\boldsymbol{\hat{z}}$ axis). We start without considering hemoglobin effects.

Import necessary packages and add the working directory to the system path.

In [1]:
import sys
import os
import dolfin
from ipygany import Scene, PolyMesh, ColorBar, IsoColor
from ipywidgets import Play, IntProgress, link, VBox
sys.path.append(os.getcwd()[:-6])

Import the model modules.

In [2]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Set a target folder for `vtk` files.

In [3]:
folder = "periodic-bcs"
path = os.path.join("../raw-data", folder)

Instance the model and run simulation.

In [4]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.generate_slab_mesh(
    dims=(200, 6, 10), elems=(100, 6, 10), save=True, periodic=True
)
model.sim_p(save=True, meshtype="slab")
model.sim_bst(final_time=1, num_steps=15, save=True, hb=False)

ValueError: Mesh type must be slab or tkd for periodicity.

Visualize using `iygany`. As an example, we start with $p$. Currently, prior knowledge of the `vtu` file is needed to visualize this. There's probably a better way to do this.

In [ ]:
mesh = PolyMesh.from_vtk(os.path.join(model.folder_path, 'p/p000000.vtu'))
iso = IsoColor(mesh, input='f_18', min=8, max=12)
scene = Scene([iso])
bar = ColorBar(iso)
VBox(children=(scene, bar))

Next we visualize $p_{\text{O}_{2}}$.

In [ ]:
mesh_dynamic = PolyMesh.from_vtk(
    os.path.join(model.folder_path, 'bst/pO2000000.vtu')
)

def load_step(change):
    mesh_dynamic.reload(
        os.path.join(model.folder_path, 'bst/pO20000{}.vtu').format(
            str(change['new']).zfill(2)
        ), reload_vertices=True
    )

play = Play(description='Step:', min=1, max=15, value=1)
play.observe(load_step, names=['value'])

progress = IntProgress(value=1, step=1, min=1, max=15)
link((progress, 'value'), (play, 'value'))

stepper = VBox((play, progress))
iso = IsoColor(mesh_dynamic, input='f_48-0', min=0, max=120)
bar = ColorBar(iso)
scene = Scene([iso])

VBox(children=(scene, stepper, bar))